Script to split document based on token length to not overload the model.

In [26]:
import os
from transformers import AutoTokenizer

def chunk_text(filename, tokenizer, max_length):
    if not os.path.exists(filename):
        print(f"The file {filename} does not exist.")
        return []

    with open(filename, 'r', encoding='utf-8') as file:
        text = file.read()

    # Add a space before each period to ensure sentence boundary is a single token
    text = text.replace(".", " .")

    tokens = tokenizer.encode(text, truncation=False, return_tensors='pt')[0]
    chunks = []

    # Initialize the start index
    start = 0

    # Loop over the tokens and create chunks
    while start < len(tokens):
        if len(tokens[start:]) > max_length:
            end = start + max_length
            # Make sure the end index is not in the middle of a sentence
            while tokens[end] != tokenizer.encode(".", add_special_tokens=False)[0]:
                end -= 1
            end += 1  # Include the period in the current chunk
        else:
            end = len(tokens)

        # Create a chunk and add it to the list of chunks
        chunk = tokens[start:end]
        chunks.append(tokenizer.decode(chunk))

        # Update the start index
        start = end

    return chunks


In [2]:
text=""""
Le handicap est défini comme : toute limitation d’activité ou restriction de participation à la vie en société subie dans son environnement par une personne en raison d’une altération substantielle, durable ou définitive d’une ou plusieurs fonctions physiques, sensorielles, mentales, cognitives ou psychiques, d’un polyhandicap ou d’un trouble de santé invalidant (article L. 114 du code de l’action sociale et des familles).

L’accessibilité numérique consiste à rendre les services de communication au public en ligne accessibles aux personnes handicapées, c’est-à-dire :

perceptibles : par exemple, faciliter la perception visuelle et auditive du contenu par l’utilisateur ; proposer des équivalents textuels à tout contenu non textuel ; créer un contenu qui puisse être présenté de différentes manières sans perte d’information ni de structure (par exemple avec une mise en page simplifiée) ;
utilisables : par exemple, fournir à l’utilisateur des éléments d’orientation pour naviguer, trouver le contenu ; rendre toutes les fonctionnalités accessibles au clavier ; laisser à l’utilisateur suffisamment de temps pour lire et utiliser le contenu ; ne pas concevoir de contenu susceptible de provoquer des crises d’épilepsie ;
compréhensibles : par exemple, faire en sorte que les pages fonctionnent de manière prévisible ; aider l’utilisateur à corriger les erreurs de saisie.
robustes : par exemple, optimiser la compatibilité avec les utilisations actuelles et futures, y compris avec les technologies d’assistance.
"""

In [27]:

tokenizer = AutoTokenizer.from_pretrained("t5-large")
filename = "vctr.txt"
chunks = chunk_text(filename, tokenizer, 412)


Token indices sequence length is longer than the specified maximum sequence length for this model (35680 > 512). Running this sequence through the model will result in indexing errors


Redis interaction

In [28]:
from transformers import T5Tokenizer, T5Model
import torch
from redis import from_url
from redis.commands.search.field import TextField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType


# Connection to the redis instance
REDIS_URL = 'redis://localhost:6379'
client = from_url(REDIS_URL)
client.ping()


tokenizer = T5Tokenizer.from_pretrained('t5-large')
model = T5Model.from_pretrained('t5-large')

def get_vector(text):
    input_ids = tokenizer.encode(text, return_tensors="pt")

    with torch.no_grad():
        # Pass the input to the encoder
        encoder_outputs = model.get_encoder()(input_ids)
        
    # Retrieve the last hidden state of the encoder
    hidden_state = encoder_outputs.last_hidden_state

    # Compute the mean over the sequence dimension
    return hidden_state.mean(dim=1).numpy().flatten().tolist()


# Builds the json with the content in natural language and the vectors
for count, chunk in enumerate(chunks):
    doc_json = {"content": chunk, "vector": get_vector(chunk)}
    
    # Creates the schema in redis based on first doc
    if count == 0:
        schema = [ VectorField('$.vector', 
                    "FLAT", 
                    {   "TYPE": 'FLOAT32', 
                        "DIM": len(doc_json['vector']), 
                        "DISTANCE_METRIC": "COSINE"
                    },  as_name='vector' ),
                    TextField('$.content', as_name='content')
                ]
        idx_def = IndexDefinition(index_type=IndexType.JSON, prefix=['doc:'])
        # try: 
        #     client.ft('idx').dropindex()
        #     print("dropped index")
        # except:
        #     pass
        # client.ft('idx').create_index(schema, definition=idx_def)
        # print("created Index")

    # Loads the document into Redis.
    # Careful with the prefix in the name as Redis use that to associate a document and an index.
    doc_name = "doc:" + str(count)
    client.json().set(doc_name, '$', doc_json)